In [1]:
import os
import json
import pandas as pd
import traceback

In [17]:
# loading env variables
from dotenv import  load_dotenv

load_dotenv()

KEY = os.getenv("OPENAI_API_KEY")


In [23]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.6)

In [25]:
# Importing libraries
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [35]:
TEMPLATE_1 = """
You are an expert in MCQ generation. Please generate {number} multiple choice questions about {subject} subject from the delimited text by ``` and question should in {tone} tone. \
Outputs should be in json object that includes the following:\
keys: question, multiple choices, and correct answer

Text to generate questions:
```{text}```




Please make sure that you don't forget about the question tone. It is really important to assess the student knowledge. If there is nothing related to {subject} subject then please show that \
\"There is no text related to {subject}\"

"""




In [33]:
# Defining prompt template

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template= TEMPLATE
)

In [34]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [38]:
TEMPLATE_2 = """
You are experienced professor. You need to evaluate the complexity of multiple choice question delimited by ``` and give a complete analysis of the quiz about {subject}.\
Only use maximum 50 words for complexity analysis. If the quiz is not suitable with the cognitive and analytical abilities of the {subject} students, update the quiz questions \
which needs to be changed and change the tone of the multiple choice questions such that perfectly aligns and fits the {subject} student abilities. Outputs should in JSON object \
that includes the following:\
keys: question, multiple choices, and correct answer

Quiz multiple choice question:
```{quiz}```

Check from an {subject} expert that the question and answer is valid and correct.
"""

In [39]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz", "subject"],
    template = TEMPLATE_2
)

In [41]:
quiz_evaluation_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [43]:
complete_quiz_chain = SequentialChain(
    chains=[quiz_chain, quiz_evaluation_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"], 
    output_variables=["quiz", "review"],
    verbose=True
 )